# Scenario 1 - Kinetic Modeling
This Jupyter-notebook will guide you through the process of visualizing and modeling your experimental data and finally uploading an EnzymeML-file to a Database. <br>
A Juypter-notebook contains text-blocks, like this one, and code-blocks, which you can run by pressing the small green triangle. Some code-blocks will generate output directly under them.<br>
You can use this notebook as a step-by-step tutorial. For each step, a short text explains what will happen when you run the code-block and what you have to do. <br>
If not stated otherwise, please run each code-cell after you read the corresponding text.<br>Before we start, we have to do some preparations in the next few blocks.

### Packages imports
Import packages used in this notebook

In [ ]:
# API
import request
import json
# Operating system for files
import os
# Jupyter widgets
from ipywidgets import widgets

### Working directory
To have one place for all your data choose a directory. When you press the button a directory manager will open and you can navigate to a folder of your choice.

### Preparing the API
For convenient API-requests later on, some preparation has to be done.

In [ ]:
url = "http://127.0.0.1:5000/"
endpoint_create = f"{url}/create"
endpoint_read = f"{url}/read"